# Classical error correction with linear codes

Information is an inherently physical object. To communicate it we must attach it to a physical manifestation such as speech, writing or actions. Any physical system can fail and when it does the information it carries can be lost or distorted.

Suppose Alice and Bob agree that Alice will turn the kitchen light on if she is working from home and turn it off otherwise. The idea is efficient but it depends entirely on the light functioning. If the bulb fails, Bob may draw the wrong conclusion despite Alice’s intention remaining unchanged. To reduce this risk Alice could instead turn on the kitchen, bedroom and bathroom lights. It is unlikely that all three bulbs fail at once, so Bob is far more likely to receive the intended message.

This is an example of classical error correction. Using multiple bulbs introduces redundancy into the communication channel so that even when parts of the physical system fail the original message can still be recovered. The goal of error correction is to introduce redundancy and structure into the communication channel so that the information remains intact despite physical imperfections.

## Encoding messages as binary strings

We can encode the light switch example using binary numbers. The switch is either on or off, which we encode as 
$1$ for on and $0$ for off. This is an example of a bit. A bit represents a physical system with two states, labelled $0$ and $1$.

Errors can occur when a bit changes state: for example a $0$ becomes a $1$ or a $1$ becomes a $0$. These are called bit-flip errors. For example, if Alice tries to send $1$ by turning the light on but the bulb fails, Bob will interpret it as a $0$ and misread the message.

A simple way to reduce errors is to increase redundancy by sending multiple copies of the same bit. We then assume that the value appearing most often is the intended message. For example, suppose that Alice sends $111$ and each bulb has probability $0.1$ of failure. Suppose that Bob receives $101$. The probability that only the middle bit flipped is $0.9 \cdot 0.1 \cdot 0.9 = 0.081$, whereas the probability the other two bits flipped is $0.1 \cdot 0.9 \cdot 0.1 = 0.009$. Therefore, it is much more likely that only one bit flipped and the intended message was $111$. In general, the most likely codeword is the one where the fewest bits have flipped, which corresponds to the majority of bits. This method is called majority decoding.

The more copies Alice sends the more likely Bob will choose the intended message. However, there is a trade-off. Sending more copies also increases the number of bits Bob must check, which takes more time and reduces efficiency. We can see this in the following example.

**Example.** Before moving on to general error correction codes we consider an implementation of the light switch example above. First we consider what happens for a single bit transmission.

In [ ]:
import random

def transmit_single_bit(bit: int, flip_probability: float) -> int:
    """ 
    Transmits one bit across a binary symmetric channel.
    Each bit has probability flip_probability of being flipped.
    """
    if random.random() < flip_probability:
        return (bit + 1) % 2
    return bit

def simulate_single_bit_channel(num_trials: int, flip_probability: float, verbose: bool = False) -> float:
    """
    Simulate num_trials transmissions through a binary symmetric channel.
    Returns the proportion of correctly received bits.
    """
    correct = 0
    for _ in range(num_trials):
        sent = random.choice([0,1])
        received = transmit_single_bit(sent, flip_probability)
        if verbose:
            print(f"Sent {sent}, received {received}, error {sent != received}.")
        if sent == received:
            correct += 1
    return float(correct/num_trials)

# Parameters
num_trials = 1000
flip_probability = 0.2

accuracy = simulate_single_bit_channel(num_trials, flip_probability)
print(f"Percentage of correct transmissions: {accuracy:.3f}")

To improve reliability, we can increase the number of light bulbs, or equivalently increase the number of bits in our binary strings. Let us consider the case with three light switches.

In [ ]:
def majority_decode(bits: list[int]):
    """
    Returns 1 if the majority are 1s, otherwise 0.
    """
    return 1 if sum(bits) > len(bits)//2 else 0


def simulate_three_bits(num_trials: int, flip_probability: float, verbose: bool = False) -> float:
    """
    Simulate transmitting a single bit as a binary string of length three.
    Each bit flips independently with probability flip_probability.
    Returns the proportion of correctly decoded messages.
    """
    correct = 0

    for _ in range(num_trials):
        sent_bit = random.choice([0, 1])
        sent_block = [sent_bit] * block_length
        received_block = [transmit_single_bit(b, flip_probability) for b in sent_block]
        decoded_bit = majority_decode(received_block)

        if verbose:
            print(f"Alice sent bit: {sent_bit}")
            print(f"Alice sent string: {sent_block}")
            print(f"Bob received string: {received_block}")
            print(f"Bob decoded bit: {decoded_bit}")
            print(f"Correct? {decoded_bit == sent_bit}\n")

        if decoded_bit == sent_bit:
            correct += 1

    return correct / num_trials

# Parameters
num_trials = 20
flip_probability = 0.2
block_length = 3

accuracy = simulate_three_bits(num_trials, flip_probability, verbose=True)
print(f"Percentage of correct interpretations: {accuracy:.3f}")

However, as the number of bits increases, the time required to decode the message also grows, since there are more bits to check.

In [ ]:
import matplotlib.pyplot as plt
import time

error_prob = 0.2 
num_trials = 50000
num_bits = range(1, 31, 2)
success_probs = []
decode_times = []

for n in num_bits:
    successes = 0
    total_time = 0

    for trial in range(num_trials):
        start_time = time.time()
        alice_intention = random.choice([0,1])
        alice_message = [alice_intention]*n
        bob_message = [transmit_single_bit(i, error_prob) for i in alice_message]
        bob_interpretation = majority_decode(bob_message)
        trial_time = time.time() - start_time
        total_time += trial_time
        if bob_interpretation == alice_intention:
            successes += 1

    success_prob = float(successes/num_trials)
    success_probs.append(success_prob)

    decode_time = float(total_time/num_trials)
    decode_times.append(decode_time)

fig, (ax1, ax2) = plt.subplots(2, sharex=True)
fig.suptitle("Majority vote decoding vs number of bits", fontsize=12)

ax1.plot(num_bits, success_probs)
ax1.set(
    ylabel="Probability of correct decoding",
    #title=f"Probability correct message is decoded using majority vote vs number of encoding bits. \n Error prob = {error_prob}."
    )
ax1.grid(visible=True)

ax2.plot(num_bits, decode_times)
ax2.set(
    xlabel="Number of bits used to encode message",
    ylabel="Time taken to decode",
    #title=f"Time taken to decode message using majority vote vs number of encoding bits. \n Error prob = {error_prob}."
    )
ax2.grid(visible=True)

plt.show

Ideally we want Bob to receive the message Alice intended with high probability as efficiently as possible.

### Encoding messages

Sometimes we want to communicate multiple pieces of information at once. Suppose Bob needs to know whether Alice is working from home today and tomorrow. Alice could use two switches, one for each day, sending the binary string $10$ if she is working today but not tomorrow.

Again to reduce the risk of errors, Alice can add redundancy. For example, she could send six bits instead of two, encoding $10$ as $111000$. Now Alice can send one of four possible binary strings: $$ 000000, 000111, 111000, 111111.$$ We can generalise this idea. Let $\mathcal{M}$ be the set of messages Alice wants to send. Choose an integer $n \in \{0,1\}^n$ and define an injective function $$ E \colon \mathcal{M} \to \{0,1\}^n. $$ Note you must choose $n$ so that $2^n$ is larger than the number of messages in $\mathcal{M}$ for such an injective function to exist. This function is shared with Bob to allow him to infer the intended message from the received string.

Of course, we still have the same issue: Bob may not receive one of the four binary strings sent by Alice. In fact, Bob could receive any $n$-bit string. Suppose Alice sends one of the four 6-bit codewords above and Bob receives $010111$. Previously we counted the most frequent value in the binary string to obtain the most likely codewood. However, now if we compare $010111$ to the possible messages sent by Alice:
| Sent binary string | Number of bit flips to get to $010111$ |
| ---                | ---                                    |
| $000000$ | 4 |
| $000111$ | 1 |
| $111000$ | 5 |
| $111111$ | 2 |

Therefore, we assume the most likely binary string that Alice sent is $000111$ and Bob takes this to be the intended message. 

This example can be generalised to obtain _error correcting codes_ by interpreting $\{0,1\}^n$ as a vector space and using linear algebra.

## Error correction codes

Fix an integer $n \in \mathbb{Z}_{>0}$. The binary strings of length $n$ are precisely the vectors in the vector space $(\mathbb{Z}/2\mathbb{Z})^n$. A _(binary) linear code_ of _length_ $n$ is a vector subspace $$\mathcal{C} \subseteq (\mathbb{Z}/2\mathbb{Z})^n.$$  An element of the code $\mathcal{C}$ is called a _codeword_. The _rank_ of a code is its dimension as a vector space over $\mathbb{Z}/2\mathbb{Z}$.

**Examples**

1. The set $\mathcal{C}_1 = \{(0,0,0), (1,1,1)\}$ is a vector subspace of $(\mathbb{Z}/2\mathbb{Z})^3$, and hence a code. The basis of the code is $\{(1,1,1)\}$. Thus, $\mathcal{C}_1$ is a code of length $3$ and rank $1$ that contains $2$ codewords.

2. The set $$\mathcal{C}_2 = \{(0,0,0),(1,1,0),(1,0,1),(0,1,1)\}$$ also forms a subspace of $(\mathbb{Z}/2\mathbb{Z})^3$, and so is a code. One basis of $\mathcal{C}_2$ is $\{(1,1,0),(1,0,1)\}$. Thus, $\mathcal{C}_2$ is a code of length $3$ and rank $2$ that contains $4$ codewords.

3. For any subset $S$ of $(\mathbb{Z}/2\mathbb{Z})^n$ the span $$\mathcal{C}(S) = \mathrm{Span}_{\mathbb{Z}/2\mathbb{Z}}(S)$$ defines a binary code of length $n$. The rank and number of codewords of $\mathcal{C}(S)$ depends on the linear relations between the elements of $S$.

Since a code is a vector subspace, a code is uniquely determined by a basis. A _generator matrix_ of a code is a matrix whose rows form a basis of the code. Note that a basis of a vector space is not unique and hence the generator matrix of a code is not unique either.

**Examples.**

1. One generator matrix for $\mathcal{C}_1$ is
$$ \begin{pmatrix} 1 & 1 & 1 \end{pmatrix}. $$

2. One generator matrix for $\mathcal{C}_2$ is
$$ \begin{pmatrix} 1 & 1 & 0 \\ 1 & 0 & 1 \end{pmatrix}. $$

In [ ]:
import random

from linearcodes import LinearCode

# C1 = {000,111}

M1 = LinearCode([[0,0,0],[1,1,1]])
print(f"\nC1 has codewords: {M1.codewords};")
print(f"  Length: {M1.length};")
print(f"  Rank: {M1.rank};")
G1 = M1.generator_matrix
print(f"  Generator matrix: \n{G1.array}.")

# C2 = {000,110,101,011}

M2 = LinearCode([[0,0,0],[1,1,0],[1,0,1],[0,1,1]])
print(f"\nC2 has codewords: {M2.codewords};")
print(f"  Length: {M2.length};")
print(f"  Rank: {M2.rank};")
G2 = M2.generator_matrix
print(f"  Generator matrix: \n{G2.array}.")

# A random code C3

def random_code(num_generators: int, code_length: int) -> LinearCode:
    """
    Creates a random list of generator vectors of length 'code_length'
    and returns the LinearCode generated by them.
    Note that this does not necessarily create a basis of the code.
    So, the number of generators may not equal the rank of the code.
    """
    generators = []
    for _ in range(num_generators):
        generator = []
        for _ in range(code_length):
            generator.append(random.choice([0,1]))
        generators.append(generator)
    return LinearCode(generators)

M3 = random_code(3,5)
codewords = M3.codewords
print(f"\nC3 has {len(codewords)} codewords:")
print(f"  {codewords};")
print(f"  Length: {M3.length};")
print(f"  Rank: {M3.rank};")
G3 = M3.generator_matrix
print(f"  Generator matrix: \n{G3.array}.")


### Dual codes

The generator matrix of a code $\mathcal{C}$ of length $n \in \mathbb{Z}_{>0}$ defines a linear map
$$G \colon (\mathbb{Z}/2\mathbb{Z})^n \to \mathcal{C}.$$
The kernel of this map is a vector subspace of $(\mathbb{Z}/2\mathbb{Z})^n$ and therefore forms another code. We call $\ker(G)$ the dual code of $\mathcal{C}$ and denote it by $\mathcal{C}^{\perp}$.

There is an equivalent characterisation of the dual code:
$$\mathcal{C}^{\perp} = \{ v \in (\mathbb{Z}/2\mathbb{Z})^n \mid c \cdot v = 0 \text{ for all } c \in \mathcal{C} \}.$$
These definitions agree because a vector $v$ satisfies $Gv = 0$ exactly when it is orthogonal to every row of the generator matrix, and these rows form a basis for $\mathcal{C}$. By the Rank Nullity Theorem, if $\mathcal{C}$ has length $n$ and rank $k$, then $\mathcal{C}^{\perp}$ has rank $n-k$. A generator matrix for $\ker(G)$ is called a parity check matrix of $\mathcal{C}$, denoted by $H$.

In summary we have:
| Code $\mathcal{C}$ | Dual code $\mathcal{C}^\perp$ |
| ---               | ---                           |
| Generator matrix $G$ | Generator matrix $H$ |
| Parity check matrix $H$ | Parity check matrix $G$ |

**Examples.**

1. The code $\mathcal{C}_1$ with generator matrix $$ G_1 = \begin{pmatrix} 1 & 1 & 1 \end{pmatrix}$$ has a parity check matrix $$ H_1 = \begin{pmatrix} 1 & 1 & 0 \\ 1 & 0 & 1 \end{pmatrix}. $$

2. The code $\mathcal{C}_2$ with generator matrix $$ G_2 = \begin{pmatrix} 1 & 1 & 0 \\ 1 & 0 & 1 \end{pmatrix}$$ has a parity check matrix  $$ H_2 = \begin{pmatrix} 1 & 1 & 1 \end{pmatrix}$$

In particular, the dual code of $\mathcal{C}_1$ is $\mathcal{C}_2$ and vice versa.



In [ ]:
import random

from linearcodes import LinearCode

# C1 = {000,111}

M1 = LinearCode([[0,0,0],[1,1,1]])
print(f"\nC1 has codewords: {M1.codewords};")
print(f"  Length: {M1.length};")
print(f"  Rank: {M1.rank};")
G1 = M1.generator_matrix
print(f"  Generator matrix: \n{G1.array}.")
H1 = M1.parity_check_matrix
print(f"  Parity check matrix: \n{H1.array}")

# C2 = {000,110,101,011}

M2 = LinearCode([[0,0,0],[1,1,0],[1,0,1],[0,1,1]])
print(f"\nC2 has codewords: {M2.codewords};")
print(f"  Length: {M2.length};")
print(f"  Rank: {M2.rank};")
G2 = M2.generator_matrix
print(f"  Generator matrix: \n{G2.array}.")
H2 = M2.parity_check_matrix
print(f"  Parity check matrix: \n{H2.array}")

# A random code C3

M3 = random_code(3,5)
codewords = M3.codewords
print(f"\nC3 has {len(codewords)} codewords:")
print(f"  {codewords};")
print(f"  Length: {M3.length};")
print(f"  Rank: {M3.rank};")
G3 = M3.generator_matrix
print(f"  Generator matrix: \n{G3.array}.")
H3 = M3.parity_check_matrix
print(f"  Parity check matrix: \n{H3.array}")


### Parity check equations

The generator matrix of the dual code $\mathcal C^{\perp}$ of a code $\mathcal{C}$ is called the _parity check matrix_ of $\mathcal{C}$ and is denoted by $H$. The parity check matrix is useful to determine vectors that are codewords of $\mathcal{C}$. Each of the rows of $H$ is a codeword in the dual code $\mathcal{C}^\perp$. Therefore, its dot product with a codeword of $\mathcal{C}$ is 0 and we have a vector $v \in (\mathbb Z/2\mathbb Z)^n$ lies in the code $\mathcal C$ if and only if $$H v^{\mathrm T} = 0.$$
Each row of $H$ is a vector of $0$'s and $1$'s. Therefore, each row defines an equation that adds up a subset of the coordinates of $v$ which we call _parity check equations_. For a vector to lie in the codespace each of these equations must given even parity.

---
**Example.** Consider the repetition code of length three with generator matrix
$G_1 = \begin{pmatrix} 1 & 1 & 1 \end{pmatrix}.$
A parity check matrix for this code is
$H_1 = \begin{pmatrix} 1 & 1 & 0 \\ 1 & 0 & 1 \end{pmatrix}.$
Since the parity check matrix has two rows, there are two parity check equations. In particular, for $v = (v_1, v_2, v_3)$ we have
$$P_1(v) = v_1 + v_2 \; \text{ and } \; P_2(v) = v_2 + v_3.$$
The vector $v$ is a codeword exactly when both sums have even parity.

---

In [ ]:
def print_parity_checks(code: LinearCode):
    """
    Print the parity check equations for the code.
    """
    parity_check_eqns = code.parity_check_eqns
    num_parity_check_eqns = code.num_parity_checks
    print(f"\nC has {num_parity_check_eqns} parity check equations given by:")
    for pc_idx, pc_eqn in enumerate(parity_check_eqns, start=1):
        terms = [f"v{bit_idx+1}" for bit_idx, bit in enumerate(pc_eqn) if bit == 1]
        readable_eqn = " + ".join(terms) if terms else "0"
        print(f" * P{pc_idx}(v) = {readable_eqn}.")

code = random_code(3, 7)
print(f"C is a code with generator matrix:\n{code.generator_matrix.array}")
print_parity_checks(code)

#### Tanner graphs

A Tanner graph is a bipartite graph associated with a parity check matrix. It has two types of vertices:
1. "Bit vertices"
2. "Check vertices"

The bit vertices correspond to bit indices in a codeword, so for a code of length $n$ there are $n$ bit vectices. The check vertices correspond to parity check equations, so for a code of length $n$ and rank $k$, there are $n-k$ check vertices. A bit vertex is connected to a check vertex when that bit appears in that parity check equation. Tanner graphs visualise the structure of a code and are used in iterative decoding algorithms.

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from linearcodes import LinearCode

def tanner_graph(code: LinearCode) -> nx.Graph:
    code_length = code.length
    parity_check_eqs = code.parity_check_eqns
    num_parity_checks = code.num_parity_checks
    G = nx.Graph()
    bit_nodes = [f"v{bit_idx+1}" for bit_idx in range(code_length)]
    check_nodes = [f"P{pc_idx+1}" for pc_idx in range(num_parity_checks)]
    G.add_nodes_from(bit_nodes, bipartite=0)
    G.add_nodes_from(check_nodes, bipartite=1)
    for pc_idx, pc_eqn in enumerate(parity_check_eqs):
        for bit_idx, bit in enumerate(pc_eqn):
            if bit == 1:
                G.add_edge(f"v{bit_idx+1}", f"P{pc_idx+1}")
    return G

def draw_tanner_graph(code: LinearCode, title: str = "Tanner Graph", bits_colour: str = "skyblue", pc_colour: str = "lightgreen"):
    G = tanner_graph(code)
    bit_nodes = [n for n, d in G.nodes(data=True) if d['bipartite'] == 0]
    pc_nodes = [n for n, d in G.nodes(data=True) if d['bipartite'] == 1]

    bit_x = range(len(bit_nodes))
    pc_x = range(len(pc_nodes))
    
    pos = {}
    for i, node in enumerate(bit_nodes):
        pos[node] = (0, bit_x[i])
    for i, node in enumerate(pc_nodes):
        pos[node] = (1, pc_x[i])
        
    nx.draw(G, pos, with_labels=True, node_color=[bits_colour if G.nodes[n]['bipartite'] == 0 else pc_colour for n in G.nodes()])
    plt.title(title)
    plt.show()

code = random_code(4,9)
print_parity_checks(code)
draw_tanner_graph(code)


#### Syndromes.

The parity check equations can be used to detect errors. Recall that an error is a bit-flip: either $0$ becomes $1$ or $1$ becomes $0$. In $(\mathbb{Z}/2\mathbb{Z})$, this is equivalent to adding $1$. Therefore, an error can be represented by an error vector $e$, with a $1$ in each position where a bit flip occurred.

If we send a codeword $c$ and receive a message $m$, there exists an error vector $e$ such that $$m=c+e.$$ Applying the parity check matrix gives $$H m^{\mathrm T} = H c^{\mathrm T} + H e^{\mathrm T} = H e^{\mathrm T}.$$ The vector $H e^{\mathrm{T}}$ depends only on the error pattern and is called the _syndrome_. Distinct error patterns can produce distinct syndromes, which allows us to detect and correct errors.


---
**Example.** The repetition code of length $3$ has a parity check matrix $H = \begin{pmatrix} 1 & 1 & 0 \\ 1 & 0 & 1 \end{pmatrix}.$ There are three possible single-bit flip errors, one for each bit. The syndromes of these error are as follows:
| Error | Error vector $e$ | Syndrome $He^\mathrm{T}$ |
| ---   | ---              | ---                      |
| First bit flips | 100 | 11 |
| Second bit flips | 010 | 10 |
| Third bit flips | 001 | 01 |

Each single-bit flip has a unique syndrome, so assuming only one error occurs, we can uniquely determine which bit was flipped.

However, if more than one error occurs, the syndrome may no longer be unique. For example, if the second and third bits flip we have that $e = 011$ and so $$ He^\mathrm{T} = 11$$ which is the same as the syndrome for a single flip of the first bit.

---


We have seen that syndromes do not always uniquely determine the errors that occur. However, there is a class of codes where like in the repetition code of length $3$ each _single_ bit error can be uniquely identified by its syndrome. That is, Hamming codes.

### Hamming codes

Hamming codes are designed to make the syndromes for single bit errors distinct. They are defined via the parity check matrix.

**Definition.** Fix an integer $r>0$. A Hamming code $\mathcal{C}$ of length $2^r-1$ has parity check matrix $H$ whose columns are all the non-zero vectors in $\{0,1\}^r$.

---
**Example.** Fix $r=3$. The Hamming code with dual code of length $2^3-1 = 7$ has parity check matrix
$$ H = 
\begin{pmatrix}
1 & 0 & 0 & 1 & 1 & 0 & 1 \\
0 & 1 & 0 & 1 & 0 & 1 & 1 \\
0 & 0 & 1 & 0 & 1 & 1 & 1
\end{pmatrix}. $$

---

### LDPC codes

An LDPC code is a linear code whose parity check matrix is sparse, meaning each parity check equation involves only a few variables. This sparsity makes it efficient to check whether a codeword contains an error. The parity checks can be visualised using a Tanner graph, a bipartite graph in which one set of nodes represents the bits in a codeword and the other set represents the parity check equations. A bit node is connected to a check node if that bit appears in the corresponding parity check equation.

To define an LDPC code, one specifies the code length (number of bits), the number of parity check equations, and the weight of each parity check (the number of bits involved in the equation).

In [ ]:
from linearcodes import RandomLDPC

ldpc_code = RandomLDPC(num_bits=9, num_parity_checks=3, parity_check_weights=[3])
ldpc_generator = ldpc_code.generator_matrix.array
print(f"LDPC code with generator matrix: \n{ldpc_generator}")
print_parity_checks(ldpc_code)
draw_tanner_graph(ldpc_code)